In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from fuzzywuzzy import fuzz

In [94]:
df_canal = pd.read_csv('./Datasets/CanalDeVenta.csv')
df_clientes = pd.read_csv('./Datasets/Clientes.csv', delimiter=';', encoding='utf-8')
df_compra = pd.read_csv('./Datasets/Compra.csv')
df_gasto = pd.read_csv('./Datasets/Gasto.csv')
df_local = pd.read_csv('./Datasets/Localidades.csv')
df_prov = pd.read_csv('./Datasets/Proveedores.csv', encoding='iso-8859-1')
df_suc = pd.read_csv('./Datasets/Sucursales.csv', delimiter=';')
df_tipo = pd.read_csv('./Datasets/TiposDeGasto.csv')
df_venta = pd.read_csv('./Datasets/Venta.csv')

# Tabla Venta

In [95]:
df_venta.describe()

,IdVenta,IdCanal,IdCliente,IdSucursal,IdEmpleado,IdProducto,Precio,Cantidad
count,46180.000000,46180.000000,46180.000000,46180.000000,46180.000000,46180.000000,4.526000e+04,45296.000000
mean,23151.534279,2.018038,1502.635232,14.782352,2488.830858,42901.264379,5.876587e+03,2.388246
std,13432.585938,0.745254,972.536404,8.876539,960.231933,83.588487,2.510651e+05,3.562045
min,1.000000,1.000000,1.000000,1.000000,1011.000000,42737.000000,3.000000e+00,1.000000
25%,11545.750000,1.000000,691.000000,7.000000,1574.000000,42833.000000,4.380000e+02,2.000000
50%,23090.500000,2.000000,1359.000000,15.000000,2468.000000,42902.000000,6.540000e+02,2.000000
75%,34635.250000,3.000000,2273.000000,23.000000,3433.000000,42974.000000,1.789000e+03,3.000000
max,47600.000000,3.000000,3407.000000,31.000000,3979.000000,43043.000000,3.373920e+07,300.000000


In [96]:
productos = [x for x in df_venta['IdProducto'].unique()]

In [97]:
def get_moda(df, element_id, referencia, target):
    """Obtiene la moda de una columna target filtrando a través de la columna referencia"""
    df_moda = df[df[referencia] == element_id].mode()
    return df_moda[target][0]

In [98]:
def input_moda(df, element_id, referencia, target):
    """Cambia todos los valores de las columnas al valor de la moda"""
    moda = get_moda(df, element_id, referencia, target)
    df.loc[(df[target] != moda) & (df[referencia] == element_id), target] = moda

In [99]:
for product in productos:
    input_moda(df_venta, product, 'IdProducto', 'Precio')

In [100]:
df_venta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46180 entries, 0 to 46179
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   IdVenta        46180 non-null  int64  
 1   Fecha          46180 non-null  object 
 2   Fecha_Entrega  46180 non-null  object 
 3   IdCanal        46180 non-null  int64  
 4   IdCliente      46180 non-null  int64  
 5   IdSucursal     46180 non-null  int64  
 6   IdEmpleado     46180 non-null  int64  
 7   IdProducto     46180 non-null  int64  
 8   Precio         46180 non-null  float64
 9   Cantidad       45296 non-null  float64
dtypes: float64(2), int64(6), object(2)
memory usage: 3.5+ MB


# Localidad

In [101]:
df_provincias = df_local.loc[:,['provincia_id','provincia_nombre']].drop_duplicates()

In [102]:
df_provincias

,provincia_id,provincia_nombre
0,6,Buenos Aires
878,10,Catamarca
1060,14,Córdoba
1628,18,Corrientes
1705,22,Chaco
1795,26,Chubut
1885,30,Entre Ríos
2061,34,Formosa
2125,38,Jujuy
2285,42,La Pampa


In [103]:
df_local

,categoria,centroide_lat,centroide_lon,departamento_id,departamento_nombre,fuente,id,localidad_censal_id,localidad_censal_nombre,municipio_id,municipio_nombre,nombre,provincia_id,provincia_nombre
0,Localidad simple,-35.033073,-60.280620,6021.0,Alberti,INDEC,6021010000,6021010,Alberti,60021.0,Alberti,ALBERTI,6,Buenos Aires
1,Localidad simple,-34.868119,-60.393971,6021.0,Alberti,INDEC,6021020000,6021020,Coronel Seguí,60021.0,Alberti,CORONEL SEGUI,6,Buenos Aires
2,Componente de localidad compuesta,-35.068014,-60.402597,6021.0,Alberti,INDEC,6021030000,6021030,Mechita,60021.0,Alberti,MECHITA,6,Buenos Aires
3,Localidad simple,-35.124382,-60.220061,6021.0,Alberti,INDEC,6021040000,6021040,Pla,60021.0,Alberti,PLA,6,Buenos Aires
4,Localidad simple,-35.109618,-60.070551,6021.0,Alberti,INDEC,6021050000,6021050,Villa Grisolía,60021.0,Alberti,VILLA GRISOLIA,6,Buenos Aires
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4137,Localidad simple,-34.688216,-58.957430,6364.0,General Rodríguez,INDEC,6364030005,6364030,General Rodríguez,60364.0,General Rodríguez,BARRIO RUTA 24 KILOMETRO 10,6,Buenos Aires
4138,Localidad simple,-53.787047,-67.713235,94008.0,Río Grande,INDEC,94008010000,94008010,Río Grande,940007.0,Río Grande,RIO GRANDE,94,"Tierra del Fuego, Antártida e Islas del Atlánt..."
4139,Localidad simple,-54.511387,-67.195804,94008.0,Río Grande,INDEC,94008020000,94008020,Tolhuin,942007.0,Tolhuin,TOLHUIN,94,"Tierra del Fuego, Antártida e Islas del Atlánt..."
4140,Localidad simple,-54.637686,-67.766941,94015.0,Ushuaia,INDEC,94015010000,94015010,Laguna Escondida,NaN,NaN,LAGUNA ESCONDIDA,94,"Tierra del Fuego, Antártida e Islas del Atlánt..."


# Tabla Clientes

In [104]:
df_clientes = df_clientes.drop(columns='col10')

In [105]:
df_clientes

,ID,Provincia,Nombre_y_Apellido,Domicilio,Telefono,Edad,Localidad,X,Y
0,1,Buenos Aires,HEBER JONI SANTANA,LAS HERAS Y BAT. 24 DE FEBRERO 4150 RINCON DE...,42-5161,58,LOMA VERDE,"-58,81850307","-34,30997088"
1,2,Buenos Aires,ANA SAPRIZA,PUEYRREDON Y DUPUY RUTA3 KM 52.500 S/N BÂº LO...,49-7578,61,SANTA ROSA,"-58,73073751","-34,93908311"
2,3,Buenos Aires,FERNANDO LUIS SARALEGUI,CALDERON DE LA BARCA 498,49-3435,15,TORRES,"-59,12794068","-34,43082199"
3,4,Buenos Aires,MANUELA SARASOLA,"RUTA 36 KM 45, 500 S/N EL PELIGRO",49-2883,29,RUTA SOL,"-58,14393954","-34,92052706"
4,5,Buenos Aires,MARIO RAÚL SARASUA,492 Y 186 S/N COLONIA URQUIZA,491-4608,34,JOSE MELCHOR ROMERO,"-58,89381","-34,9444471"
...,...,...,...,...,...,...,...,...,...
2379,2668,Córdoba,MARIA STELLA RIVAS,CALLE PUBLICA SANTA ELENA COLONIA TIROLESA - ...,4902756,23,COLONIA TIROLESA,"-64,7419272","-31,25881725"
2380,2669,Córdoba,ANA GABRIELA RIVAS,CALLE PUBLICA S/NÂº LA ARCADIA,153580992,28,LA ARCADIA,"-64,7696886","-31,53338396"
2381,2670,Córdoba,JAVIER ALONSO RIVEIRO,AVENIDA LIBERTADOR SAN MARTIN S/NÂº BOUWER,4993041,57,BOUWER,"-64,19298343","-31,55786125"
2382,2671,Córdoba,JUAN MANUEL RODRIGUEZ,CALLE PUBLICA S/NÂ° EL GATEADO BARRIO EL GATE...,152768918,36,EL GATEADO,"-64,11681227","-31,32123448"


In [106]:
def get_apellidos(df, columna):
    apellidos = []
    for nombre in df[columna]:
        if isinstance(nombre, str):
            nombre = nombre.lower()
            if ', ' in nombre:
                apellidos.append(nombre.split(', ')[0])
            else:
                words = nombre.split(' ')
                if len(words) > 3:
                    last = words[-2:]
                    apellidos.append(' '.join(last))
                else:
                    apellidos.append(words[-1])
        else:
            apellidos.append(float('NaN'))
    return apellidos

In [107]:
def get_names(element):
    '''Obtengo sólo los nombres'''
    if isinstance(element, str):
        element = element.lower()
        if ', ' in element:
            return element.split(', ')[1]

        names_last = element.split(' ')
        if len(names_last) > 3:
            names = names_last[:-2]
        else:
            names = names_last[:-1]

        return ' '.join(names)

In [108]:
def nombre_apellido(df):
    '''Creo una columna de apellidos y nombres por separado'''
    apellidos = get_apellidos(df, 'Nombre_y_Apellido')
    df.insert(3, 'Apellido', apellidos)
    df['Nombre_y_Apellido'] = df['Nombre_y_Apellido'].apply(get_names)
    df.rename(columns={'Nombre_y_Apellido':'Nombre'}, inplace=True)


In [109]:
nombre_apellido(df_clientes)

In [110]:
df_clientes.head()

,ID,Provincia,Nombre,Apellido,Domicilio,Telefono,Edad,Localidad,X,Y
0,1,Buenos Aires,heber joni,santana,LAS HERAS Y BAT. 24 DE FEBRERO 4150 RINCON DE...,42-5161,58,LOMA VERDE,"-58,81850307","-34,30997088"
1,2,Buenos Aires,ana,sapriza,PUEYRREDON Y DUPUY RUTA3 KM 52.500 S/N BÂº LO...,49-7578,61,SANTA ROSA,"-58,73073751","-34,93908311"
2,3,Buenos Aires,fernando luis,saralegui,CALDERON DE LA BARCA 498,49-3435,15,TORRES,"-59,12794068","-34,43082199"
3,4,Buenos Aires,manuela,sarasola,"RUTA 36 KM 45, 500 S/N EL PELIGRO",49-2883,29,RUTA SOL,"-58,14393954","-34,92052706"
4,5,Buenos Aires,mario raúl,sarasua,492 Y 186 S/N COLONIA URQUIZA,491-4608,34,JOSE MELCHOR ROMERO,"-58,89381","-34,9444471"


In [111]:
def provincia_to_id(element):
    '''Obtengo el Id de la provincia y reemplazo su nombre'''
    if isinstance(element, str):
        mask = df_provincias[df_provincias['provincia_nombre'].apply(lambda x: fuzz.partial_ratio(x,element)) > 80] # Aplico la función de fuzz, que implementa Levinshtein, para obtener el id de la provincia
        return mask['provincia_id'].values.tolist()[0]
    else:
        return element

In [112]:
df_clientes['Provincia'] = df_clientes['Provincia'].apply(provincia_to_id)
df_clientes.rename(columns={'Provincia':'IdProvincia'}, inplace=True)

In [113]:
df_clientes['IdProvincia'] = df_clientes['IdProvincia'].astype('Int64')

In [114]:
def replace_comma(stringed):
    try:
        with_comma = stringed.replace(',','.')
        return float(with_comma)
    except:
        return float('NaN')

In [115]:
df_clientes.X = df_clientes.X.apply(replace_comma)
df_clientes.Y = df_clientes.Y.apply(replace_comma)

In [184]:
df_clientes

,ID,IdProvincia,Nombre,Apellido,Domicilio,Telefono,Edad,Localidad,X,Y
0,1,6,heber joni,santana,LAS HERAS Y BAT. 24 DE FEBRERO 4150 RINCON DE...,42-5161,58,LOMA VERDE,-58.818503,-34.309971
1,2,6,ana,sapriza,PUEYRREDON Y DUPUY RUTA3 KM 52.500 S/N BÂº LO...,49-7578,61,SANTA ROSA,-58.730738,-34.939083
2,3,6,fernando luis,saralegui,CALDERON DE LA BARCA 498,49-3435,15,TORRES,-59.127941,-34.430822
3,4,6,manuela,sarasola,"RUTA 36 KM 45, 500 S/N EL PELIGRO",49-2883,29,RUTA SOL,-58.143940,-34.920527
4,5,6,mario raúl,sarasua,492 Y 186 S/N COLONIA URQUIZA,491-4608,34,JOSE MELCHOR ROMERO,-58.893810,-34.944447
...,...,...,...,...,...,...,...,...,...,...
2379,2668,14,maria stella,rivas,CALLE PUBLICA SANTA ELENA COLONIA TIROLESA - ...,4902756,23,COLONIA TIROLESA,-64.741927,-31.258817
2380,2669,14,ana gabriela,rivas,CALLE PUBLICA S/NÂº LA ARCADIA,153580992,28,LA ARCADIA,-64.769689,-31.533384
2381,2670,14,javier alonso,riveiro,AVENIDA LIBERTADOR SAN MARTIN S/NÂº BOUWER,4993041,57,BOUWER,-64.192983,-31.557861
2382,2671,14,juan manuel,rodriguez,CALLE PUBLICA S/NÂ° EL GATEADO BARRIO EL GATE...,152768918,36,EL GATEADO,-64.116812,-31.321234


In [186]:
df_venta

,IdVenta,Fecha,Fecha_Entrega,IdCanal,IdCliente,IdSucursal,IdEmpleado,IdProducto,Precio,Cantidad
0,1,2018-03-09,2018-03-17,3,969,13,1674,42817,813.12,2.0
1,2,2018-12-28,2018-12-29,2,884,13,1674,42795,543.18,3.0
2,3,2016-03-28,2016-03-31,2,1722,13,1674,42837,430.32,1.0
3,4,2017-10-23,2017-10-24,3,2876,13,1674,42834,818.84,2.0
4,5,2017-11-22,2017-11-25,2,678,13,1674,42825,554.18,3.0
...,...,...,...,...,...,...,...,...,...,...
46175,47596,2020-11-30,2020-12-09,1,193,26,3032,42850,279.00,2.0
46176,47597,2020-11-30,2020-12-07,3,333,26,3193,42799,1536.04,2.0
46177,47598,2020-11-30,2020-12-07,1,2389,27,3667,42916,1356.00,2.0
46178,47599,2020-11-30,2020-12-04,1,1067,29,3836,42935,2456.00,1.0
